In [335]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn import linear_model as lm
from sklearn.feature_selection import r_regression
from sklearn.metrics import r2_score

In [247]:
import warnings
warnings.filterwarnings('ignore')

In [248]:
approval = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-FKWVTTFtJT6u56e0bqdfoMcXvDO1DUChsJ3jQAMB2lZk2SMqVfmg7dGjclTYkYWz-Pm5lfcLPjp4/pub?output=csv')
# approval = approval.reset_index()
# approval = approval.rename({'index':'poll_id'}, axis=1)
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,tracking,approve,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp
0,Donald Trump (2nd term),Adults,Morning Consult,NaN,6/3/2025,6/9/2025,45000.0,A,1.718826,1.523125,True,49.81,44.14,5.670002,47.97300,47.58297,0.390030,https://pro.morningconsult.com/trackers/global...,6/9/2025,20:00:30 15 Jun 2025
1,Donald Trump (2nd term),Immigration,The Washington Post,NaN,6/10/2025,6/10/2025,1015.0,A,1.521513,1.434895,NaN,37.00,52.00,-15.000000,39.03083,50.58693,-11.556100,https://www.washingtonpost.com/documents/e75c5...,6/10/2025,20:00:38 15 Jun 2025
2,Donald Trump (2nd term),Trade,Quinnipiac University,NaN,6/5/2025,6/9/2025,1265.0,RV,1.377684,1.377684,NaN,38.00,57.00,-19.000000,38.22469,56.85495,-18.630260,https://poll.qu.edu/poll-release?releaseid=3924,6/9/2025,20:00:40 15 Jun 2025
3,Donald Trump (2nd term),Economy,Quinnipiac University,NaN,6/5/2025,6/9/2025,1265.0,RV,1.377684,1.299254,NaN,40.00,56.00,-16.000000,40.85153,55.44485,-14.593320,https://poll.qu.edu/poll-release?releaseid=3924,6/9/2025,20:00:36 15 Jun 2025
4,Donald Trump (2nd term),Immigration,J.L. Partners,NaN,6/10/2025,6/11/2025,1807.0,RV,1.272316,1.272316,NaN,46.00,37.00,9.000000,45.07502,39.66180,5.413219,https://www.dailymail.co.uk/news/article-14803...,6/11/2025,20:00:38 15 Jun 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512,Donald Trump (2nd term),Economy,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,42.00,36.00,6.000000,44.51486,36.40462,8.110237,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,20:00:36 15 Jun 2025
1513,Donald Trump (2nd term),Strong,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,29.00,29.00,0.000000,33.28094,29.36967,3.911270,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,20:00:32 15 Jun 2025
1514,Donald Trump (2nd term),Trade,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.996709,0.000000,NaN,38.00,40.00,-2.000000,40.17828,40.23442,-0.056141,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,20:00:40 15 Jun 2025
1515,Donald Trump (2nd term),All polls,InsiderAdvantage,NaN,1/20/2025,1/20/2025,800.0,RV,1.093180,0.000000,NaN,56.00,39.00,17.000000,53.69289,40.80989,12.883000,https://insideradvantage.com/first-insideradva...,1/20/2025,20:00:23 15 Jun 2025


In [249]:
approval = approval[~approval['subgroup'].isin(['Weak', 'Strong', 'Economy', 'Cost', 'Immigration', 'Trade', 'Adults', 'Voters'])]

In [250]:
approval.to_csv('trump_approval_polls.csv')

In [251]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'enddate', 'samplesize', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp'], dtype=object)

In [252]:
approval.shape

(330, 20)

In [253]:
# dates = approval['Dates'].str.extract(r'(\d{1,2}\/\d{1,2}) \- (\d{1,2}\/\d{1,2}), (\d{4})')
# dates

In [254]:
# dates['end_date'] = dates[1] + '/' + dates[2]
# dates['end_date'] = pd.to_datetime(dates['end_date'])
# dates

In [255]:
# approval['Dates'] = dates['end_date']
# approval.head()

In [256]:
# samples = approval['Sample'].str.extract(r'[0-9,]{3,5} (\w{1,2})@@(\d{3,4})')
# samples

In [257]:
# approval['population'] = samples[0]
# approval['sample_size'] = samples[1].astype(int)
# approval.head()

In [258]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [259]:
ratings['Pollster'].values

array(['AtlasIntel', 'Marquette University Law School',
       'The Washington Post', 'Research & Polling Inc.',
       'The New York Times/Siena College', 'Cygnal', 'SurveyUSA',
       'Siena College', 'Beacon Research/Shaw & Co. Research',
       'Alaska Survey Research', 'OnMessage Inc.', 'TIPP Insights',
       'Emerson College', 'Landmark Communications',
       'Monmouth University', 'Data Orbital',
       'University of North Florida', 'MassINC Polling Group',
       'Marist College', 'Public Policy Institute of California',
       'Public Policy Polling', 'University of California Berkeley',
       'CBS News/The New York Times', 'Christopher Newport University',
       'Mason-Dixon Polling & Strategy', 'Strategic Vision LLC',
       'Suffolk University', 'Stockton University',
       'Field Research Corp. (Field Poll)', 'Quinnipiac University',
       'Los Angeles Times',
       'Hart Research Associates/Public Opinion Strategies',
       'Public Opinion Strategies', 'Susquehan

In [260]:
ratings[ratings['Pollster'].str.contains('North Star')]#.sum()

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
75,North Star Opinion Research,B/C@@8,0.043339,R+0.63@@-0.62697512,13,R+
425,Univision/Latino Decisions/North Star Opinion ...,B/C@@8,0.830409,D+1.06@@1.0564657,7,D+


In [261]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [262]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(330, 20)

In [263]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='inner')
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),All polls,Quinnipiac University,NaN,6/5/2025,6/9/2025,1265.0,RV,1.377684,1.063965,...,54.00,-16.000000,40.19531,53.24822,-13.052910,https://poll.qu.edu/poll-release?releaseid=3924,6/9/2025,20:00:23 15 Jun 2025,Quinnipiac University,-0.290484
1,Donald Trump (2nd term),All polls,J.L. Partners,NaN,6/10/2025,6/11/2025,1807.0,RV,1.021452,1.021452,...,46.00,-4.000000,42.71870,48.98553,-6.266830,https://www.dailymail.co.uk/news/article-14803...,6/11/2025,20:00:23 15 Jun 2025,J.L. Partners,0.197911
2,Donald Trump (2nd term),All polls,Morning Consult,NaN,6/3/2025,6/9/2025,45000.0,A,1.157798,0.815103,...,44.14,5.670002,48.63794,45.55445,3.083488,https://pro.morningconsult.com/trackers/global...,6/9/2025,20:00:23 15 Jun 2025,Morning Consult,0.584994
3,Donald Trump (2nd term),All polls,SurveyMonkey,NBC News,5/30/2025,6/10/2025,19410.0,A,1.234728,0.812979,...,55.00,-10.000000,44.99593,53.61444,-8.618511,https://s3.documentcloud.org/documents/2597610...,6/10/2025,20:00:23 15 Jun 2025,SurveyMonkey,1.047575
4,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,6/12/2025,1500.0,LV,0.842307,0.740202,...,45.00,8.000000,50.11303,46.51521,3.597820,https://www.rasmussenreports.com/public_conten...,6/12/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,Donald Trump (2nd term),All polls,Research Co.,NaN,1/22/2025,1/24/2025,1001.0,A,0.951747,0.000000,...,46.00,4.000000,50.08036,45.41092,4.669441,https://researchco.ca/2025/01/27/immigration-usa/,1/24/2025,20:00:23 15 Jun 2025,Research Co.,0.632258
311,Donald Trump (2nd term),All polls,AtlasIntel,NaN,1/21/2025,1/23/2025,1882.0,A,1.910964,0.000000,...,49.70,0.399998,49.91399,48.33871,1.575279,https://cdn.atlasintel.org/ed3f5238-e823-495e-...,1/23/2025,20:00:23 15 Jun 2025,AtlasIntel,-0.957740
312,Donald Trump (2nd term),All polls,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.00,18.000000,53.83838,41.25202,12.586360,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,20:00:23 15 Jun 2025,RMG Research,0.278023
313,Donald Trump (2nd term),All polls,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,...,41.00,6.000000,48.96566,40.04660,8.919060,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,20:00:23 15 Jun 2025,Ipsos,0.442898


In [264]:
approval.shape

(315, 22)

In [265]:
approval['Pollster'].value_counts()

Pollster
Rasmussen Reports                                     101
YouGov                                                 39
Morning Consult                                        25
RMG Research                                           21
Ipsos                                                  14
J.L. Partners                                          10
Global Strategy Group/GBAO (Navigator Research)         7
TIPP Insights                                           7
Echelon Insights                                        6
Quinnipiac University                                   5
Emerson College                                         5
Harris Insights & Analytics                             5
Gallup                                                  5
AtlasIntel                                              5
SSRS                                                    4
American Research Group                                 4
McLaughlin & Associates                                 4
Marqu

In [266]:
# approval[~approval['tracking'].isna()]

In [267]:
approval['tracking'].value_counts()

tracking
True    105
Name: count, dtype: int64

In [268]:
# full_approval = approval.copy()

In [269]:
approval = approval[(~approval['samplesize'].isna())]
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),All polls,Quinnipiac University,NaN,6/5/2025,6/9/2025,1265.0,RV,1.377684,1.063965,...,54.00,-16.000000,40.19531,53.24822,-13.052910,https://poll.qu.edu/poll-release?releaseid=3924,6/9/2025,20:00:23 15 Jun 2025,Quinnipiac University,-0.290484
1,Donald Trump (2nd term),All polls,J.L. Partners,NaN,6/10/2025,6/11/2025,1807.0,RV,1.021452,1.021452,...,46.00,-4.000000,42.71870,48.98553,-6.266830,https://www.dailymail.co.uk/news/article-14803...,6/11/2025,20:00:23 15 Jun 2025,J.L. Partners,0.197911
2,Donald Trump (2nd term),All polls,Morning Consult,NaN,6/3/2025,6/9/2025,45000.0,A,1.157798,0.815103,...,44.14,5.670002,48.63794,45.55445,3.083488,https://pro.morningconsult.com/trackers/global...,6/9/2025,20:00:23 15 Jun 2025,Morning Consult,0.584994
3,Donald Trump (2nd term),All polls,SurveyMonkey,NBC News,5/30/2025,6/10/2025,19410.0,A,1.234728,0.812979,...,55.00,-10.000000,44.99593,53.61444,-8.618511,https://s3.documentcloud.org/documents/2597610...,6/10/2025,20:00:23 15 Jun 2025,SurveyMonkey,1.047575
4,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,6/12/2025,1500.0,LV,0.842307,0.740202,...,45.00,8.000000,50.11303,46.51521,3.597820,https://www.rasmussenreports.com/public_conten...,6/12/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,Donald Trump (2nd term),All polls,Research Co.,NaN,1/22/2025,1/24/2025,1001.0,A,0.951747,0.000000,...,46.00,4.000000,50.08036,45.41092,4.669441,https://researchco.ca/2025/01/27/immigration-usa/,1/24/2025,20:00:23 15 Jun 2025,Research Co.,0.632258
311,Donald Trump (2nd term),All polls,AtlasIntel,NaN,1/21/2025,1/23/2025,1882.0,A,1.910964,0.000000,...,49.70,0.399998,49.91399,48.33871,1.575279,https://cdn.atlasintel.org/ed3f5238-e823-495e-...,1/23/2025,20:00:23 15 Jun 2025,AtlasIntel,-0.957740
312,Donald Trump (2nd term),All polls,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.00,18.000000,53.83838,41.25202,12.586360,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,20:00:23 15 Jun 2025,RMG Research,0.278023
313,Donald Trump (2nd term),All polls,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,...,41.00,6.000000,48.96566,40.04660,8.919060,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,20:00:23 15 Jun 2025,Ipsos,0.442898


In [270]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [271]:
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,2.00
1,D,1.02
2,D,0.63
3,D,0.79
4,R,0.60
...,...,...
93,R,2.44
94,R,0.45
95,R,3.23
96,R,4.94


In [272]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[0] * house_eff[1]
house_eff

,0,1,house_effect
0,-1,2.00,-2.00
1,1,1.02,1.02
2,1,0.63,0.63
3,1,0.79,0.79
4,-1,0.60,-0.60
...,...,...,...
93,-1,2.44,-2.44
94,-1,0.45,-0.45
95,-1,3.23,-3.23
96,-1,4.94,-4.94


In [273]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-2.00
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.02
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.63
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.79
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.60
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.44
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.45
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.23


In [274]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['approve_adj'] = df['approve'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['disapprove_adj'] = df['disapprove'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['approve_adj'] = df['approve']
        df['disapprove_adj'] = df['disapprove']
    return df

In [275]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj
4,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,6/12/2025,1500.0,LV,0.842307,0.740202,...,50.11303,46.51521,3.597820,https://www.rasmussenreports.com/public_conten...,6/12/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,52.360,45.640
17,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,5/30/2025,6/5/2025,1500.0,LV,0.324773,0.144533,...,47.11303,50.51521,-3.402180,https://www.rasmussenreports.com/public_conten...,6/5/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,49.360,49.640
20,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/5/2025,6/11/2025,1500.0,LV,0.117946,0.096451,...,50.11303,46.51521,3.597820,https://www.rasmussenreports.com/public_conten...,6/11/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,52.360,45.640
24,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/2/2025,6/6/2025,1500.0,LV,0.155384,0.088643,...,48.11303,49.51521,-1.402180,https://www.rasmussenreports.com/public_conten...,6/6/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,50.360,48.640
25,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/4/2025,6/10/2025,1500.0,LV,0.115884,0.087662,...,49.11303,48.51521,0.597820,https://www.rasmussenreports.com/public_conten...,6/10/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,51.360,47.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Donald Trump (2nd term),All polls,Remington Research Group,Humanity for Freedom Foundation,3/15/2025,3/18/2025,800.0,LV,0.999693,0.000000,...,49.16528,48.60749,0.557789,https://www.humanityforfreedomfoundation.org/f...,3/18/2025,20:00:23 15 Jun 2025,Remington Research Group,0.322523,49.295,48.705
206,Donald Trump (2nd term),All polls,North Star Opinion Research,Defending Democracy Together,3/13/2025,3/17/2025,1000.0,RV,1.158492,0.000000,...,46.28109,50.75347,-4.472382,https://www.defendingdemocracytogether.org/app...,3/17/2025,20:00:23 15 Jun 2025,North Star Opinion Research,0.043339,46.000,51.000
266,Donald Trump (2nd term),All polls,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2/19/2025,1000.0,LV,1.404286,0.000000,...,49.49333,48.59719,0.896137,https://onmessageinc.com/wp-content/uploads/20...,2/19/2025,20:00:23 15 Jun 2025,OnMessage Inc.,-0.600249,48.225,50.775
279,Donald Trump (2nd term),All polls,SurveyUSA,NaN,2/13/2025,2/16/2025,2000.0,A,1.824873,0.000000,...,50.21747,45.74694,4.470528,https://www.surveyusa.com/client/PollReport.as...,2/16/2025,20:00:23 15 Jun 2025,SurveyUSA,-0.732433,50.765,45.235


In [276]:
approval.shape

(315, 22)

In [277]:
approval = approval_adj.copy()
approval.shape

(315, 24)

In [278]:
approval['poll_id'] = pd.factorize(approval['pollster'] + approval['enddate'])[0]
approval.head(7)

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj,poll_id
4,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,6/12/2025,1500.0,LV,0.842307,0.740202,...,46.51521,3.59782,https://www.rasmussenreports.com/public_conten...,6/12/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,52.36,45.64,0
17,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,5/30/2025,6/5/2025,1500.0,LV,0.324773,0.144533,...,50.51521,-3.40218,https://www.rasmussenreports.com/public_conten...,6/5/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,49.36,49.64,1
20,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/5/2025,6/11/2025,1500.0,LV,0.117946,0.096451,...,46.51521,3.59782,https://www.rasmussenreports.com/public_conten...,6/11/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,52.36,45.64,2
24,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/2/2025,6/6/2025,1500.0,LV,0.155384,0.088643,...,49.51521,-1.40218,https://www.rasmussenreports.com/public_conten...,6/6/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,50.36,48.64,3
25,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/4/2025,6/10/2025,1500.0,LV,0.115884,0.087662,...,48.51521,0.59782,https://www.rasmussenreports.com/public_conten...,6/10/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,51.36,47.64,4
27,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/3/2025,6/9/2025,1500.0,LV,0.114323,0.079398,...,49.51521,-1.40218,https://www.rasmussenreports.com/public_conten...,6/9/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,50.36,48.64,5
29,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,5/23/2025,5/29/2025,1500.0,LV,0.316798,0.060198,...,47.51521,2.59782,https://www.rasmussenreports.com/public_conten...,5/29/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,52.36,46.64,6


In [279]:
approval['population'] = approval['population'].str.strip(' ')

In [280]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id', keep=False)]
    ndup = data[~data.duplicated(subset='poll_id', keep=False)]
    # print('dup', dup, '\n ndup', ndup)
    dup_a = dup[dup['population'].isin(['A'])]
    # print('dup_a', dup_a)
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id', keep=False)]
    ndup = df[~df.duplicated(subset='poll_id', keep=False)]
    dup_lv = dup[dup['population'].isin(['RV'])]
    # print('dup_rv', dup_lv, 'ndup', ndup)
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [281]:
approval = approval.rename({'enddate':'end_date', 'samplesize':'sample_size', 'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [282]:
approval['population'].value_counts().index.values

array(['LV', 'RV', 'A', 'V'], dtype=object)

In [283]:
approval.shape, pipeline(approval).shape

((315, 25), (305, 25))

In [284]:
pipeline(approval)

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
4,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,2025-06-12,1500.0,LV,0.842307,0.740202,...,46.51521,3.597820,https://www.rasmussenreports.com/public_conten...,6/12/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,52.360,45.640,0
17,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,5/30/2025,2025-06-05,1500.0,LV,0.324773,0.144533,...,50.51521,-3.402180,https://www.rasmussenreports.com/public_conten...,6/5/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,49.360,49.640,1
20,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/5/2025,2025-06-11,1500.0,LV,0.117946,0.096451,...,46.51521,3.597820,https://www.rasmussenreports.com/public_conten...,6/11/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,52.360,45.640,2
24,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/2/2025,2025-06-06,1500.0,LV,0.155384,0.088643,...,49.51521,-1.402180,https://www.rasmussenreports.com/public_conten...,6/6/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,50.360,48.640,3
25,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/4/2025,2025-06-10,1500.0,LV,0.115884,0.087662,...,48.51521,0.597820,https://www.rasmussenreports.com/public_conten...,6/10/2025,20:00:23 15 Jun 2025,Rasmussen Reports,0.087015,51.360,47.640,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Donald Trump (2nd term),All polls,Remington Research Group,Humanity for Freedom Foundation,3/15/2025,2025-03-18,800.0,LV,0.999693,0.000000,...,48.60749,0.557789,https://www.humanityforfreedomfoundation.org/f...,3/18/2025,20:00:23 15 Jun 2025,Remington Research Group,0.322523,49.295,48.705,305
206,Donald Trump (2nd term),All polls,North Star Opinion Research,Defending Democracy Together,3/13/2025,2025-03-17,1000.0,RV,1.158492,0.000000,...,50.75347,-4.472382,https://www.defendingdemocracytogether.org/app...,3/17/2025,20:00:23 15 Jun 2025,North Star Opinion Research,0.043339,46.000,51.000,306
266,Donald Trump (2nd term),All polls,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2025-02-19,1000.0,LV,1.404286,0.000000,...,48.59719,0.896137,https://onmessageinc.com/wp-content/uploads/20...,2/19/2025,20:00:23 15 Jun 2025,OnMessage Inc.,-0.600249,48.225,50.775,307
279,Donald Trump (2nd term),All polls,SurveyUSA,NaN,2/13/2025,2025-02-16,2000.0,A,1.824873,0.000000,...,45.74694,4.470528,https://www.surveyusa.com/client/PollReport.as...,2/16/2025,20:00:23 15 Jun 2025,SurveyUSA,-0.732433,50.765,45.235,308


In [285]:
# approval[approval['pollster']=='Harris Insights & Analytics']

In [286]:
full_approval = approval.copy()

In [287]:
approval = approval[approval['tracking'].isna()]

In [288]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs',
       'Trafalgar Group', 'OnMessage Inc.', 'McLaughlin & Associates',
       'North Star Opinion Research', 'Trafalgar Group/Insider Advantage'],
      dtype=object)

In [289]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
7,Donald Trump (2nd term),All polls,YouGov,Economist,6/6/2025,6/9/2025,1533.0,A,0.707933,0.576948,...,52.44852,-8.69722,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/9/2025,20:00:23 15 Jun 2025,YouGov,0.047635,43.25,51.75,100
10,Donald Trump (2nd term),All polls,YouGov,CBS News,6/4/2025,6/6/2025,2428.0,A,0.772760,0.485450,...,55.44852,-9.69722,https://www.cbsnews.com/news/deportation-immig...,6/6/2025,20:00:23 15 Jun 2025,YouGov,0.047635,45.25,54.75,101
16,Donald Trump (2nd term),All polls,YouGov,Economist,5/30/2025,6/2/2025,1610.0,A,0.603392,0.227726,...,49.44852,-3.69722,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/2/2025,20:00:23 15 Jun 2025,YouGov,0.047635,45.25,48.75,102
22,Donald Trump (2nd term),All polls,YouGov,Economist,5/23/2025,5/26/2025,1660.0,A,0.570159,0.093593,...,52.44852,-7.69722,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/26/2025,20:00:23 15 Jun 2025,YouGov,0.047635,44.25,51.75,103
23,Donald Trump (2nd term),All polls,YouGov,Yahoo News,5/22/2025,5/27/2025,1560.0,A,0.556163,0.091296,...,54.44852,-12.69722,https://www.yahoo.com/news/poll-most-americans...,5/27/2025,20:00:23 15 Jun 2025,YouGov,0.047635,41.25,53.75,104


In [290]:
approval[approval['pollster']=='Harris Insights & Analytics']

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
31,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,5/14/2025,5/15/2025,1903.0,RV,1.305367,0.056145,...,49.8658,-4.093437,https://harvardharrispoll.com/key-results-may-5/,5/15/2025,20:00:23 15 Jun 2025,Harris Insights & Analytics,0.162447,47.105,47.895,235
152,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,4/9/2025,4/10/2025,2286.0,RV,0.950531,0.000000,...,47.8658,-1.093437,https://harvardharrispoll.com/wp-content/uploa...,4/10/2025,20:00:23 15 Jun 2025,Harris Insights & Analytics,0.162447,48.105,45.895,237
181,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,3/26/2025,3/27/2025,2746.0,RV,1.124949,0.000000,...,47.8658,-0.093437,https://harvardharrispoll.com/key-results-marc...,3/27/2025,20:00:23 15 Jun 2025,Harris Insights & Analytics,0.162447,49.105,45.895,238
262,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,2/19/2025,2/20/2025,2443.0,RV,1.382092,0.000000,...,44.8658,5.906563,https://harvardharrispoll.com/wp-content/uploa...,2/20/2025,20:00:23 15 Jun 2025,Harris Insights & Analytics,0.162447,52.105,42.895,239


In [291]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'end_date', 'sample_size', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp', 'Pollster', 'pred_plus_minus', 'approve_adj',
       'disapprove_adj', 'poll_id'], dtype=object)

In [292]:
approval_polls = approval[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'approve_adj', 'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id']]
approval_polls

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id
7,YouGov,Economist,6/6/2025,6/9/2025,1533.0,A,43.0,52.0,43.250,51.750,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,100
10,YouGov,CBS News,6/4/2025,6/6/2025,2428.0,A,45.0,55.0,45.250,54.750,-10.0,https://www.cbsnews.com/news/deportation-immig...,0.047635,101
16,YouGov,Economist,5/30/2025,6/2/2025,1610.0,A,45.0,49.0,45.250,48.750,-4.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,102
22,YouGov,Economist,5/23/2025,5/26/2025,1660.0,A,44.0,52.0,44.250,51.750,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,103
23,YouGov,Yahoo News,5/22/2025,5/27/2025,1560.0,A,41.0,54.0,41.250,53.750,-13.0,https://www.yahoo.com/news/poll-most-americans...,0.047635,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Public Religion Research Institute,NaN,2/28/2025,3/20/2025,5025.0,A,43.0,54.0,43.000,54.000,-11.0,https://www.prri.org/wp-content/uploads/2025/0...,0.103864,303
198,Public Opinion Strategies,Paragon Health Institute,2/25/2025,3/2/2025,1000.0,RV,51.0,49.0,48.930,51.070,2.0,https://paragoninstitute.org/medicaid/paragon-...,-0.246974,304
203,Remington Research Group,Humanity for Freedom Foundation,3/15/2025,3/18/2025,800.0,LV,50.0,48.0,49.295,48.705,2.0,https://www.humanityforfreedomfoundation.org/f...,0.322523,305
279,SurveyUSA,NaN,2/13/2025,2/16/2025,2000.0,A,51.0,45.0,50.765,45.235,6.0,https://www.surveyusa.com/client/PollReport.as...,-0.732433,308


In [293]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    exp_weights = 0.1**(delta/(40))
    state_pivot['time_weights'] =  0 * linear_weights + 1 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'RV':
            return 0.8
        elif population == 'LV':
            return 0.6
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

    # Gather the weights together
    state_pivot['total_weights'] = state_pivot['sample_size_weights'] * state_pivot['time_weights'] * state_pivot['quality_weights'] * state_pivot['population_weights']
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [294]:
approval_polls['sample_size'].isna().any()

False

In [295]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
7,YouGov,Economist,6/6/2025,2025-06-09,1533.0,A,43.0,52.0,43.250,51.750,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,100,0.005319,0.707946,0.964021,1.0,0.061022
10,YouGov,CBS News,6/4/2025,2025-06-06,2428.0,A,45.0,55.0,45.250,54.750,-10.0,https://www.cbsnews.com/news/deportation-immig...,0.047635,101,0.006694,0.595662,0.964021,1.0,0.064616
16,YouGov,Economist,5/30/2025,2025-06-02,1610.0,A,45.0,49.0,45.250,48.750,-4.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,102,0.005451,0.473151,0.964021,1.0,0.041796
22,YouGov,Economist,5/23/2025,2025-05-26,1660.0,A,44.0,52.0,44.250,51.750,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,103,0.005535,0.316228,0.964021,1.0,0.028364
23,YouGov,Yahoo News,5/22/2025,2025-05-27,1560.0,A,41.0,54.0,41.250,53.750,-13.0,https://www.yahoo.com/news/poll-most-americans...,0.047635,104,0.005365,0.334965,0.964021,1.0,0.029126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Public Religion Research Institute,NaN,2/28/2025,2025-03-20,5025.0,A,43.0,54.0,43.000,54.000,-11.0,https://www.prri.org/wp-content/uploads/2025/0...,0.103864,303,0.007441,0.006683,0.923213,1.0,0.000772
198,Public Opinion Strategies,Paragon Health Institute,2/25/2025,2025-03-02,1000.0,RV,51.0,49.0,48.930,51.070,2.0,https://paragoninstitute.org/medicaid/paragon-...,-0.246974,304,0.004296,0.002371,1.209225,0.8,0.000166
203,Remington Research Group,Humanity for Freedom Foundation,3/15/2025,2025-03-18,800.0,LV,50.0,48.0,49.295,48.705,2.0,https://www.humanityforfreedomfoundation.org/f...,0.322523,305,0.003842,0.005957,0.780286,0.6,0.000180
279,SurveyUSA,NaN,2/13/2025,2025-02-16,2000.0,A,51.0,45.0,50.765,45.235,6.0,https://www.surveyusa.com/client/PollReport.as...,-0.732433,308,0.006075,0.001059,1.756652,1.0,0.000190


In [296]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['approve_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['disapprove_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [297]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Approve,Disapprove
0,2025-01-23,51.310566,45.653535
1,2025-01-24,51.237797,45.330152
2,2025-01-25,51.237797,45.330152
3,2025-01-26,50.599984,45.294367
4,2025-01-27,49.443088,45.451175
...,...,...,...
139,2025-06-11,44.545320,51.298431
140,2025-06-12,45.056531,50.933808
141,2025-06-13,45.056531,50.933808
142,2025-06-14,45.056531,50.933808


In [298]:
polls_df.shape

(181, 19)

In [299]:
polls_df.head(3)

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
7,YouGov,Economist,6/6/2025,2025-06-09,1533.0,A,43.0,52.0,43.25,51.75,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,100,0.005319,0.707946,0.964021,1.0,0.061022
10,YouGov,CBS News,6/4/2025,2025-06-06,2428.0,A,45.0,55.0,45.25,54.75,-10.0,https://www.cbsnews.com/news/deportation-immig...,0.047635,101,0.006694,0.595662,0.964021,1.0,0.064616
16,YouGov,Economist,5/30/2025,2025-06-02,1610.0,A,45.0,49.0,45.25,48.75,-4.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,102,0.005451,0.473151,0.964021,1.0,0.041796


In [300]:
polls_df['net_adj'] = polls_df['approve_adj'] - polls_df['disapprove_adj']

In [301]:
display_df = polls_df[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'net',
                      'total_weights', 'url']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 'total_weights':'Weight', 'startdate':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population']
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df['Pollster'] = '[' + display_df['Pollster'] + '](' + display_df['URL'] + ')'
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net
0,"<a href=""https://napolitannews.org/jobapproval...",2025-06-02,2025-06-12,3000 RV,0.067983,52.0,46.0,6.000000
1,"<a href=""https://www.dailymail.co.uk/news/arti...",2025-06-10,2025-06-11,1807 RV,0.052976,42.0,46.0,-4.000000
2,"<a href=""https://s3.documentcloud.org/document...",2025-05-30,2025-06-10,19410 A,0.018759,45.0,55.0,-10.000000
3,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-06-06,2025-06-09,1533 A,0.061022,43.0,52.0,-9.000000
4,"<a href=""https://navigatorresearch.org/wp-cont...",2025-06-05,2025-06-09,1000 RV,0.008180,42.0,55.0,-13.000000
...,...,...,...,...,...,...,...,...
176,"<a href=""https://echeloninsights.com/in-the-ne...",2025-01-22,2025-01-24,1024 LV,0.000015,51.0,43.0,8.000000
177,"<a href=""https://x.com/ScottWRasmussen/status/...",2025-01-20,2025-01-23,3000 RV,0.000021,57.0,39.0,18.000000
178,"<a href=""https://cdn.atlasintel.org/ed3f5238-e...",2025-01-21,2025-01-23,1882 A,0.000055,50.1,49.7,0.399998
179,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,0.000013,47.0,41.0,6.000000


In [302]:
display_df.to_csv('polls_display_table.csv')

In [303]:
approval_std

,Date,Approve,Disapprove
0,2025-01-23,3.702450,5.577774
1,2025-01-24,3.406119,5.195595
2,2025-01-25,3.406119,5.195595
3,2025-01-26,3.666745,4.786089
4,2025-01-27,3.694118,4.404223
...,...,...,...
139,2025-06-11,3.234287,3.154961
140,2025-06-12,3.651339,3.331632
141,2025-06-13,3.651339,3.331632
142,2025-06-14,3.651339,3.331632


In [304]:
approval_avg['Net'] = approval_avg['Approve'] - approval_avg['Disapprove']
approval_avg

,Date,Approve,Disapprove,Net
0,2025-01-23,51.310566,45.653535,5.657031
1,2025-01-24,51.237797,45.330152,5.907644
2,2025-01-25,51.237797,45.330152,5.907644
3,2025-01-26,50.599984,45.294367,5.305617
4,2025-01-27,49.443088,45.451175,3.991913
...,...,...,...,...
139,2025-06-11,44.545320,51.298431,-6.753111
140,2025-06-12,45.056531,50.933808,-5.877277
141,2025-06-13,45.056531,50.933808,-5.877277
142,2025-06-14,45.056531,50.933808,-5.877277


In [305]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Approve_x'] + 1.96*approval_trendline['Approve_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Approve_x'] - 1.96*approval_trendline['Approve_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Disapprove_x'] + 1.96*approval_trendline['Disapprove_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Disapprove_x'] - 1.96*approval_trendline['Disapprove_y']
approval_trendline = approval_trendline.rename({
    'Approve_x':'approve_mean',
    'Disapprove_x':'disapprove_mean',
}, axis='columns').drop(['Approve_y', 'Disapprove_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-23,51.310566,45.653535,58.567369,44.053763,56.585972,34.721099
1,2025-01-24,51.237797,45.330152,57.913791,44.561802,55.513519,35.146785
2,2025-01-25,51.237797,45.330152,57.913791,44.561802,55.513519,35.146785
3,2025-01-26,50.599984,45.294367,57.786804,43.413164,54.675102,35.913632
4,2025-01-27,49.443088,45.451175,56.683559,42.202617,54.083453,36.818897
...,...,...,...,...,...,...,...
139,2025-06-11,44.545320,51.298431,50.884522,38.206118,57.482156,45.114707
140,2025-06-12,45.056531,50.933808,52.213155,37.899908,57.463807,44.403809
141,2025-06-13,45.056531,50.933808,52.213155,37.899908,57.463807,44.403809
142,2025-06-14,45.056531,50.933808,52.213155,37.899908,57.463807,44.403809


In [306]:
# Export approval ratings
approval_trendline.to_csv('approval_stats.csv')

In [307]:
polls_df.shape

(181, 20)

In [308]:
line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
line.update_traces(hovertemplate=None)
line.update_layout(hovermode='x unified')
# scatter.update_traces(hovertemplate=None, hoverinfo='skip')
fig_approve_CI = go.Figure([
    go.Scatter(
        name='Approve CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Approve CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])
fig_disapprove_CI = go.Figure([
    go.Scatter(
        name='Disapprove CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Disapprove CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])

fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
fig.update_layout(
    title='President Trump Approval Rating',
    xaxis_title='Date',
    yaxis_title='%',
    legend_title='Legend',
    hovermode='x unified'
)
fig.show()

In [309]:
line = px.line(approval_avg, x='Date', y=['Net'])
scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net'])
                     #trendline='lowess', trendline_options=dict(frac=0.2))
fig = go.Figure(data=line.data + scatter.data)
fig.add_hline(y=0, line_dash='dash')
fig.show()

In [310]:
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net', 'net_adj'])
# fig = go.Figure(data=scatter.data)
# fig.add_hline(y=0, line_dash='dash')
# fig.show()

In [311]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['approve_adj'] - data['disapprove_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [312]:
net_approval = net_col(approval, datetime.date(2025, 1, 23))
net_approval

,Date,net,net_std
0,2025-01-23,5.657031,8.630764
1,2025-01-24,5.907644,7.966501
2,2025-01-25,5.907644,7.966501
3,2025-01-26,5.305617,7.611568
4,2025-01-27,3.991913,7.074258
...,...,...,...
139,2025-06-11,-6.753111,5.460349
140,2025-06-12,-5.877277,6.189091
141,2025-06-13,-5.877277,6.189091
142,2025-06-14,-5.877277,6.189091


In [313]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-23,5.657031,8.630764,22.573328,-11.259267
1,2025-01-24,5.907644,7.966501,21.521987,-9.706698
2,2025-01-25,5.907644,7.966501,21.521987,-9.706698
3,2025-01-26,5.305617,7.611568,20.224291,-9.613057
4,2025-01-27,3.991913,7.074258,17.857460,-9.873633
...,...,...,...,...,...
139,2025-06-11,-6.753111,5.460349,3.949173,-17.455395
140,2025-06-12,-5.877277,6.189091,6.253342,-18.007896
141,2025-06-13,-5.877277,6.189091,6.253342,-18.007896
142,2025-06-14,-5.877277,6.189091,6.253342,-18.007896


In [314]:
net_approval.to_csv('net_approval.csv')

### Auxiliary Graph: Individual Pollsters

In [315]:
approval_polls.columns.values

array(['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size',
       'population', 'approve', 'disapprove', 'approve_adj',
       'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id'],
      dtype=object)

In [316]:
appr = full_approval.copy()
appr['net_adj'] = appr['approve'] - appr['disapprove']
pollsters = appr['pollster'].value_counts().index.values
pollsters

array(['Rasmussen Reports', 'YouGov', 'Morning Consult', 'RMG Research',
       'Ipsos', 'J.L. Partners',
       'Global Strategy Group/GBAO (Navigator Research)', 'TIPP Insights',
       'Echelon Insights', 'Gallup', 'AtlasIntel',
       'Quinnipiac University', 'Harris Insights & Analytics',
       'Emerson College', 'SSRS', 'American Research Group',
       'McLaughlin & Associates', 'co/efficient', 'Civiqs',
       'John Zogby Strategies', 'Cygnal',
       'Marquette University Law School', 'Clarity Campaign Labs',
       'SurveyMonkey', 'InsiderAdvantage', 'Marist College',
       'Hart Research Associates/Public Opinion Strategies',
       'Pew Research Center', 'Fabrizio Lee & Associates/Impact Research',
       'Blueprint Polling', 'Beacon Research/Shaw & Co. Research',
       'Public Religion Research Institute', 'SurveyUSA',
       'OnMessage Inc.', 'North Star Opinion Research',
       'Remington Research Group', 'Public Opinion Strategies',
       'OH Predictive Insights', 

In [317]:
def approval_today(data: pd.DataFrame):
    # date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
#     yes_averages = []
#     no_averages = []
#     yes_stdevs = []
#     no_stdevs = []
    pipelined_df = avgs_pipeline(data, datetime.datetime.today())
    yes_avg = np.sum(pipelined_df['approve'] * pipelined_df['total_weights'])
    no_avg = np.sum(pipelined_df['disapprove'] * pipelined_df['total_weights'])
    yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve'] - yes_avg)**2))
    no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove'] - no_avg)**2))
    net_avg = np.sum(pipelined_df['net'] * pipelined_df['total_weights'])
    net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net'] - net_avg)**2))
    
    #return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [318]:
def indiv_pollster_avg(pollster: str):
    df = appr[appr['pollster'] == pollster]
    yes_avg, no_avg, yes_std, no_std, net_avg, net_std = approval_today(df)
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [319]:
def get_ppm(pollster: str):
    df = ratings[ratings['Pollster'] == pollster]
    ppm = df['Predictive Plus-Minus'].values[0]
    
    return ppm

In [320]:
appr.shape

(315, 26)

In [321]:
def get_num_polls(pollster: str):
    df = appr[appr['pollster'] == pollster]
    num_polls = df.shape[0]
    return num_polls

In [322]:
get_ppm('YouGov')

0.047635015

In [323]:
indiv_pollster_avg('YouGov')

(43.59728850373825,
 52.11675983211063,
 1.4952315509641103,
 2.1987595602849916,
 -8.51947132837239,
 2.763382385521666)

In [324]:
indiv_pollster_avg('RMG Research')

(50.58458172302433,
 47.448954696068554,
 1.517861366761977,
 1.7812044224106738,
 3.1356270269557793,
 3.2007997648849957)

In [325]:
nets = []
net_stdevs = []
ppms = []
nums = []

for p in pollsters:
    _, _, _, _, net, net_std = indiv_pollster_avg(p)
    ppm = get_ppm(p)
    num_polls = get_num_polls(p)
    
    nets.append(net)
    net_stdevs.append(net_std)
    ppms.append(ppm)
    nums.append(num_polls)

In [326]:
# nets

In [327]:
indiv_avgs = pd.DataFrame({'Pollster': pollsters, 'Net Approval': nets, 'Net Stdev': net_stdevs, 'Predictive Plus-Minus': ppms, 'Number of Approval Polls Conducted': nums})
indiv_avgs = indiv_avgs.sort_values(['Net Approval'], ascending=True)
indiv_avgs

,Pollster,Net Approval,Net Stdev,Predictive Plus-Minus,Number of Approval Polls Conducted
27,Pew Research Center,-18.737919,1.965333,0.132230,2
14,SSRS,-16.700812,3.726438,0.966835,4
11,Quinnipiac University,-15.842158,0.938909,-0.290484,5
15,American Research Group,-13.187664,1.853512,0.818141,4
6,Global Strategy Group/GBAO (Navigator Research),-12.044599,1.672573,0.804988,7
41,The New York Times/Siena College,-12.000000,0.000000,-0.763947,1
40,NewsNation,-12.000000,0.000000,0.614459,1
31,Public Religion Research Institute,-11.000000,0.000000,0.103864,1
25,Marist College,-10.710159,1.394588,-0.436610,2
4,Ipsos,-10.277746,2.426948,0.442898,14


In [328]:
indiv_avgs.to_csv('indiv_pollster_avgs.csv')

In [331]:
reg = lm.LinearRegression()
reg.fit(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
reg.coef_, reg.intercept_

(array([-1.28506191]), -4.882801401707026)

In [338]:
r = r_regression(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
r

array([-0.11025513])